## 工夫した点
生存予測で使用した訓練セットは、年齢, 客室番号の欠損値が目立った。  
客室番号の処理が分からなかったため、年齢のみ補填することにした。  
年齢の補填に平均値, 中央値を入れるかと考えたが、外国では名前に敬称をつけるようなので、それを利用した。

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

/Users/sudatakamune/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/sudatakamune/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# 訓練データ
train = pd.read_csv('./dataset/train.csv')

In [3]:
# 訓練データ上から５個
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# 特徴「家族の人数」を追加
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1

# 特徴量
PassengerId 乗客ID, Survived 生存, Pclass 階級, Name 名前, Sex 性別, Age 年齢, SibSp 夫婦・兄弟, Parch 親・子  
Ticket　チケット番号, Fare 運賃, Cabin 客室番号, Embarked 寄港

In [5]:
# それぞれの欠損値の数
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
FamilySize       0
dtype: int64

In [6]:
# 名前(Name)から階級(Title)を抽出
train['Title'] = train['Name'].str.extract('([A-Za-z]+)\.', expand=False)

## 敬称(Title) から年齢を補填  
敬称の種類  
Cap:船長, Col:少佐・大佐, Countess:女伯爵, Don:貴人・高位聖職者, Dr:医者・博士号, Jonkheer:男爵, Lady:貴婦人 ,Major:少佐, Master:少年, Miss:未婚女性, Mlle:未婚女性, Mme:既婚女性, Mr:成人男性, Mrs:既婚女性, Ms:女性, Rev:牧師, Sir:騎士

In [7]:
Counter(train['Title'])

Counter({'Capt': 1,
         'Col': 2,
         'Countess': 1,
         'Don': 1,
         'Dr': 7,
         'Jonkheer': 1,
         'Lady': 1,
         'Major': 2,
         'Master': 40,
         'Miss': 182,
         'Mlle': 2,
         'Mme': 1,
         'Mr': 517,
         'Mrs': 125,
         'Ms': 1,
         'Rev': 6,
         'Sir': 1})

In [8]:
# 年齢に欠損のある敬称
Counter(train[train['Age'].isnull()].Title)

Counter({'Dr': 1, 'Master': 4, 'Miss': 36, 'Mr': 119, 'Mrs': 17})

In [9]:
# 欠損値のある敬称の平均値
Dr_age = train.query('Title == "Dr"').Age.mean()
Master_age = train.query('Title == "Master"').Age.mean()
Miss_age = train.query('Title == "Miss"').Age.mean()
Mr_age = train.query('Title == "Mr"').Age.mean()
Mrs_age = train.query('Title == "Mrs"').Age.mean()

In [10]:
# 同敬称の年齢の平均値を補填
train.loc[(train["Title"].values == "Dr") & (train["Age"].isnull()), "Age"] = Dr_age
train.loc[(train["Title"].values == "Master") & (train["Age"].isnull()), "Age"] = Master_age
train.loc[(train["Title"].values == "Miss") & (train["Age"].isnull()), "Age"] = Miss_age
train.loc[(train["Title"].values == "Mr") & (train["Age"].isnull()), "Age"] = Mr_age
train.loc[(train["Title"].values == "Mrs") & (train["Age"].isnull()), "Age"] = Mrs_age

敬称の種類が多すぎるのでまとめる。  
残す敬称は、少年(Master), 未婚女性(Miss), 既婚女性(Mrs), 男性(Mr), その他(Others)

In [11]:
# 近い敬称をまとめる
train['Title'] = train['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Countess', 'Sir', 'Jonkheer', 'Lady', 'Capt', 'Don'], 'Others')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')
train['Title'] = train['Title'].replace('Mlle', 'Miss')

## 必要ない特徴の除外、欠損値の補填、数値化  
乗客ID, 名前, 夫婦・兄弟の数, 親・子の数, チケット番号, 客室番号を除外  
寄港地の欠損値はSを補填  
性別・寄港地・敬称をそれぞれ数値化  

In [12]:
train = train.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis = 1)
train = train.fillna({'Embarked' :'S'})
train = train.replace({
    'male' : '0', 'female' : '1', 
    'S' : '0', 'C' : '1', 'Q' : '2',
    'Master' : '0', 'Miss' : '1', 'Mrs' : '2', 'Mr' : '3', 'Others' : '4'
})

In [13]:
# 前処理した訓練データ
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,FamilySize,Title
0,0,3,0,22.0,7.2500,0,2,3
1,1,1,1,38.0,71.2833,1,2,2
2,1,3,1,26.0,7.9250,0,1,1
3,1,1,1,35.0,53.1000,0,2,2
4,0,3,0,35.0,8.0500,0,1,3


# パラメータの設定

In [14]:
svm_params = [
    {'C' : [1, 10, 100, 1000], 
     'kernel' : ['linear', 'rbf', 'poly', 'sigmoid'],
     'degree': [2, 3, 4], 
     'gamma': [0.1, 0.01, 0.001, 0.0001]}
]
RandomForest_params = [
    {'n_estimators' : [3, 10, 100, 1000, 10000], 
     'n_jobs': [-1]}
]
LogisticRegression_params = [
    {'C' : [0.001, 0.01, 0.1, 1, 10, 100]}
]

In [15]:
train_data, test_data, train_labels, test_labels = train_test_split(
    train.iloc[:, 1:], 
    train["Survived"], 
    test_size=0.2, 
    random_state=0
)

In [ ]:
# svmのパラメータ
score = 'f1'
clf_svm = GridSearchCV(
    SVC(), 
    svm_params, 
    cv=5, 
    scoring='%s_weighted' % score
) 
clf_svm.fit(train_data, train_labels) 
clf_svm.best_params_

/Users/sudatakamune/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/sudatakamune/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/sudatakamune/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/sudatakamune/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


SVMの最適パラメータは、C=1, degree=2, kernel='linear'

In [ ]:
# RandomForestのパラメータ
score = 'f1'
clf_RandomForest = GridSearchCV(
    RandomForestClassifier(), 
    RandomForest_params, 
    cv=5, 
    scoring='%s_weighted' % score 
) 
clf_RandomForest.fit(train_data, train_labels) 
clf_RandomForest.best_params_

RandomForestの最適パラメータは、n_estimators = 10, n_jobs = -1

In [ ]:
# LogisticRegressionのパラメータ
score = 'f1'
clf_LogisticRegression = GridSearchCV(
    LogisticRegression(), 
    LogisticRegression_params, 
    cv=5, 
    scoring='%s_weighted' % score
) 
clf_LogisticRegression.fit(train_data, train_labels) 
clf_LogisticRegression.best_params_

# LogisticRegressionの最適パラメータは、C = 100

# モデル構築

それぞれの最適なパラメータを使用し、モデルの構築を行う。

In [ ]:
# モデルの設定
models = []
models.append(("SVM", SVC(C  = 10, gamma = 0.01)))
models.append(("Random Forest Classifier", RandomForestClassifier()))
models.append(("Logistic Regression", LogisticRegression()))

In [ ]:
# 5 Fold Cross Validation
kf = KFold(n_splits=5) 
names = []
score = []
for name, model in models:
    names.append(name)
    scores = []
    for train_split, test_split in kf.split(train):
        train_data, test_data = train.iloc[train_split], train.iloc[test_split]
        train_labels, test_labels = train_data["Survived"], test_data["Survived"]
        model.fit(train_data, train_labels)
        result = model.predict(test_data)
        acc = accuracy_score(result, test_labels)
        scores.append(acc)
        print(model.score(train_data, train_labels))
        print(model.score(test_data, test_labels))
    score.append(np.mean(scores))

#  各モデルの検証結果

In [ ]:
for i in range(len(names)):
    print("{}:""{}".format(names[i], score[i]))

Grid_Searchで最適パラメータを設定したものの、精度が100%になってしまった。

## さらに精度を上げるために   
さらに精度を上げるのに必要な前処理として、以下の点があげられる。  
・何人のグループで乗るか。(救命ボートに乗る際、グループ人数が少なければ潜り込めたかもしれない)  
・どこの客室に滞在していたか。(甲板に近い客室であれば、素早く避難することができたであろう)  